In [1]:
import numpy as np
from numpy import exp, sqrt, abs
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
plt.rcParams['font.size'] = 13

In [2]:
dname="DATA"; fname=dname+'/x_RBM_2024_lesson.dat'
v = np.loadtxt(fname, delimiter=" ",dtype=int)
v0 = np.copy(v)
N = len(v)
L = len(v[1])
SPINS = False

if SPINS:
    GAP=2
    v = 2*v - 1
    vmin=-1
else:
    GAP=1
    vmin=0

print(f'each of N={N} data has L={L} digits')

for i in range(min(12,N)):
    if SPINS: print(v0[i],"\n->",v[i])
    else: print(v[i])
print('...')

each of N=10000 data has L=12 digits
[0 0 1 1 0 0 1 0 0 0 1 1]
[0 0 1 1 0 0 0 1 0 0 0 0]
[1 1 0 0 1 0 0 0 0 1 0 0]
[0 0 1 1 0 1 1 1 1 0 0 0]
[1 0 1 0 1 1 1 1 1 1 0 0]
[0 0 1 0 1 1 0 0 1 1 0 0]
[1 1 1 0 0 0 1 1 0 1 1 1]
[1 1 1 1 0 0 1 0 1 1 0 0]
[1 1 0 0 0 0 1 1 0 0 1 1]
[1 1 1 0 1 1 0 0 1 0 0 0]
[0 0 0 1 0 1 0 0 0 0 1 1]
[1 1 1 1 1 1 0 1 1 1 0 0]
...


In [3]:
Ms = [1,2,3,4,5,6]
sigma = {}
w = {}
a = {}
b = {}

for M in Ms:
    sigma[M] = sqrt(4. / float(L + M))
    np.random.seed(12345)
    w[M] = sigma[M] * np.random.randn(L,M)
    a[M] = sigma[M] * np.random.randn(L)
    b[M] = np.zeros(M)
    print("w[M] =", w[M])
    print("a[M] =", a[M])
    print("b[M] =", b[M])
w0 = w.copy()
a0 = a.copy()
b0 = b.copy()

w[M] = [[-0.11355138]
 [ 0.26566996]
 [-0.28813276]
 [-0.30826371]
 [ 1.09041887]
 [ 0.77292249]
 [ 0.05153602]
 [ 0.15628465]
 [ 0.42657697]
 [ 0.69139759]
 [ 0.55868813]
 [-0.7190141 ]]
a[M] = [ 0.15253791  0.12697802  0.75046323  0.49170253 -1.11030861 -0.20626113
  0.92580867 -0.24327472 -0.29939469  0.26458368  1.80218983 -0.56647511]
b[M] = [0.]
w[M] = [[-0.10942085  0.25600598]
 [-0.27765167 -0.29705034]
 [ 1.05075391  0.74480675]
 [ 0.04966135  0.15059965]
 [ 0.41105985  0.66624739]
 [ 0.53836536 -0.69285933]
 [ 0.14698921  0.12235908]
 [ 0.72316447  0.47381641]
 [-1.06992015 -0.1987582 ]
 [ 0.89213155 -0.23442538]
 [-0.28850394  0.25495921]
 [ 1.73663357 -0.54586907]]
a[M] = [-3.08466139e-01  6.63456126e-02  1.61753753e-01  2.79967947e-01
  5.02599612e-04  7.18296549e-01 -3.81405303e-01 -4.44270254e-01
 -1.26694211e+00 -9.94618479e-01 -4.60094183e-01  2.99410253e-01]
b[M] = [0. 0.]
w[M] = [[-1.05710581e-01  2.47325276e-01 -2.68236999e-01]
 [-2.86977895e-01  1.01512472e+00  7.1

In [4]:
def activate(v_in,wei,bias,DE,info=False):
    act = np.dot(v_in, wei) + bias
    n = np.shape(act)
    prob = 1. / (1. + exp(-DE*act))
    v_out = np.full(n, vmin, dtype=int) # a list on -1's or 0's
    v_out[np.random.random_sample(n) < prob] = 1 # activate the 1's with probability prob
    if info:
        print('input=', v_in)
        print('act=',act)
        print('prob=',prob)
        print('output=',v_out)
    return v_out

k = 0
for M in Ms:
    print(f"Activating for M = {M}")
    activate(v[M], w[M], b[M], GAP, vmin, info=True)

Activating for M = 1


TypeError: activate() got multiple values for argument 'info'

In [ ]:
def plotgraph(epoch=0):
    fig, ax = plt.subplots(1,1 , figsize=(7, 3.3))
    ax.tick_params(left=False,bottom=False)
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    
    A=1./max(w.max(),-w.min())
    for i in range(L):
        for j in range(M):
            ex, ey, col = (y1[i],y2[j]),(x1[i],x2[j]),mycolor(w[i][j])
            ax.plot(ex, ey, col, zorder=1, alpha=A*abs(w[i][j]))
    # Scatter plot on top of lines
    #A=300./(a.max()+b.max())
    A=300.
    for i in range(L):
        ax.scatter(y1[i],x1[i], s=A*abs(a[i]), zorder=2, c=mycolor(a[i]))
    for j in range(M):
        ax.scatter(y2[j], x2[j], s=min(300,A*abs(b[j])), zorder=2, c=mycolor(b[j]), marker="s")
    ax.set_title(f'>0 red, <0 blue, epoch={epoch}')
    ax.text(-0.51,0.85,"hidden\nlayer")
    plt.show()